In [1]:
# CELL 1: Import All Required Libraries
import cv2
import numpy as np
import os
import tensorflow as tf
from tqdm import tqdm
from collections import defaultdict, Counter
from skimage import color, exposure
from sklearn.ensemble import IsolationForest
from scipy.stats.mstats import winsorize
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import gc

print("All libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")

2025-12-27 09:32:37.266229: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766827957.454775      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766827957.510007      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766827957.946183      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766827957.946231      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766827957.946234      55 computation_placer.cc:177] computation placer alr

All libraries imported successfully!
TensorFlow version: 2.19.0


In [2]:
# CELL 2: Color Normalization Functions

def reinhard_normalize(img, target_means=[148.60, 41.56, 81.44], 
                       target_stds=[41.56, 15.57, 18.44]):
    """
    Reinhard color normalization to handle stain variation.
    """
    import warnings
    warnings.filterwarnings('ignore', message='.*CIE-LAB.*')
    
    img_float = img.astype(np.float32) / 255.0 if img.dtype == np.uint8 else img
    img_lab = color.rgb2lab(img_float)
    
    for i in range(3):
        mean = np.mean(img_lab[:, :, i])
        std = np.std(img_lab[:, :, i])
        
        if std < 1e-6:
            std = 1.0
        
        img_lab[:, :, i] = ((img_lab[:, :, i] - mean) / std) * target_stds[i] + target_means[i]
    
    img_lab[:, :, 0] = np.clip(img_lab[:, :, 0], 0, 100)
    img_lab[:, :, 1] = np.clip(img_lab[:, :, 1], -127, 127)
    img_lab[:, :, 2] = np.clip(img_lab[:, :, 2], -127, 127)
    
    img_normalized = color.lab2rgb(img_lab)
    img_normalized = np.clip(img_normalized, 0, 1)
    img_normalized = (img_normalized * 255).astype(np.uint8)
    
    return img_normalized





In [3]:
def macenko_normalize(img):
    """
    Macenko-style normalization for histopathology images.
    """
    img_float = img.astype(np.float32) / 255.0
    img_float = np.maximum(img_float, 1e-6)
    od = -np.log10(img_float)
    
    h, w, c = od.shape
    od_flat = od.reshape(-1, 3)
    
    od_threshold = 0.15
    tissue_mask = np.all(od_flat > od_threshold, axis=1)
    
    if np.sum(tissue_mask) < 100:
        return img
    
    od_tissue = od_flat[tissue_mask]
    od_mean = np.mean(od_tissue, axis=0, keepdims=True)
    od_std = np.std(od_tissue, axis=0, keepdims=True) + 1e-6
    
    target_mean = np.array([[0.7, 0.5, 0.6]])
    target_std = np.array([[0.15, 0.12, 0.13]])
    
    od_normalized = (od_flat - od_mean) / od_std * target_std + target_mean
    od_normalized = np.maximum(od_normalized, 0)
    
    img_normalized = 10 ** (-od_normalized)
    img_normalized = np.clip(img_normalized, 0, 1)
    img_normalized = img_normalized.reshape(h, w, c)
    img_normalized = (img_normalized * 255).astype(np.uint8)
    
    return img_normalized

In [4]:
def combined_normalize(img, method='both'):
    if method == 'none':
        return img
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    if method == 'reinhard':
        img_normalized = reinhard_normalize(img_rgb)
    elif method == 'macenko':
        img_normalized = macenko_normalize(img_rgb)
    elif method == 'both':
        img_normalized = macenko_normalize(img_rgb)
        img_normalized = reinhard_normalize(img_normalized)
    else:
        raise ValueError(f"Unknown normalization method: {method}")
    
    img_normalized = cv2.cvtColor(img_normalized, cv2.COLOR_RGB2BGR)
    return img_normalized

print("Normalization functions defined!")

Normalization functions defined!


In [5]:
def generate_tissue_mask(img, morph_kernel_size=5):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    kernel = np.ones((morph_kernel_size, morph_kernel_size), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    return mask

print("Tissue mask function defined!")

Tissue mask function defined!


In [6]:
# CELL 4: Multi-Scale Patch Extraction

PATCH_CONFIGS = [
    {'size': 224, 'stride': 112, 'scale': 1.0, 'name': '20x'},
    {'size': 224, 'stride': 112, 'scale': 0.5, 'name': '40x'},
]

TISSUE_THRESHOLD = 0.7


def extract_patches_multiscale(img, mask, save_dir, slide_id, config, oversample_factor=1):
    patch_size = config['size']
    stride = config['stride']
    scale = config['scale']
    mag_name = config['name']
    
    if scale != 1.0:
        new_h = int(img.shape[0] * scale)
        new_w = int(img.shape[1] * scale)
        img_scaled = cv2.resize(img, (new_w, new_h))
        mask_scaled = cv2.resize(mask, (new_w, new_h))
    else:
        img_scaled = img
        mask_scaled = mask
    
    h, w, _ = img_scaled.shape
    patch_count = 0
    
    for oversample_idx in range(oversample_factor):
        offset_y = np.random.randint(0, stride // 2) if oversample_idx > 0 else 0
        offset_x = np.random.randint(0, stride // 2) if oversample_idx > 0 else 0
        
        for y in range(offset_y, h - patch_size + 1, stride):
            for x in range(offset_x, w - patch_size + 1, stride):
                mask_patch = mask_scaled[y:y+patch_size, x:x+patch_size]
                
                if np.mean(mask_patch > 0) < TISSUE_THRESHOLD:
                    continue
                
                patch = img_scaled[y:y+patch_size, x:x+patch_size]
                name = f"{slide_id}_{mag_name}_x{x}_y{y}_os{oversample_idx}.png"
                cv2.imwrite(os.path.join(save_dir, name), patch)
                patch_count += 1
    
    return patch_count


def process_wsi_enhanced(image_path, label_name, output_root, normalization_method='both',
                         oversample_factor=1):
    slide_id = os.path.splitext(os.path.basename(image_path))[0]
    img = cv2.imread(image_path)
    
    if img is None:
        print(f"Warning: Could not read {image_path}")
        return 0
    
    if normalization_method != 'none':
        img = combined_normalize(img, method=normalization_method)
    
    mask = generate_tissue_mask(img)
    save_dir = os.path.join(output_root, label_name)
    os.makedirs(save_dir, exist_ok=True)
    
    total_patches = 0
    for config in PATCH_CONFIGS:
        patches = extract_patches_multiscale(
            img, mask, save_dir, slide_id, config, oversample_factor
        )
        total_patches += patches
    
    return total_patches

print("Patch extraction functions defined!")


Patch extraction functions defined!


In [8]:
# CELL 5: Process All Images and Extract Patches
DATASET_ROOT = "/kaggle/input/her2-sish-dataset"
PATCH_ROOT = "/kaggle/working/patches"

classes = {
    "Amplified Samples": "Amplified",
    "Non-Amplified Samples": "Non_Amplified",
    "ROI_Normal": "ROI_Normal"
}

oversample_factors = {
    "Amplified": 1,
    "Non_Amplified": 1,
    "ROI_Normal": 3
}

NORMALIZATION_METHOD = 'both'  # Options: 'reinhard', 'macenko', 'both', 'none'

patch_stats = defaultdict(int)

print(f"Using normalization method: {NORMALIZATION_METHOD}")

for folder, label in classes.items():
    folder_path = os.path.join(DATASET_ROOT, folder)
    if not os.path.exists(folder_path):
        print(f"Warning: Folder not found: {folder_path}")
        continue
        
    img_list = [f for f in os.listdir(folder_path) 
                if f.lower().endswith((".png", ".jpg"))]
    
    oversample = oversample_factors[label]
    
    for img_name in tqdm(img_list, desc=f"Processing {label} (OS={oversample})", ncols=100):
        num_patches = process_wsi_enhanced(
            os.path.join(folder_path, img_name),
            label,
            PATCH_ROOT,
            normalization_method=NORMALIZATION_METHOD,
            oversample_factor=oversample
        )
        patch_stats[label] += num_patches

print("\nPatch Extraction Statistics")
for label, count in patch_stats.items():
    print(f"{label}: {count} patches")



Using normalization method: both


Processing ROI_Normal (OS=3): 100%|███████████████████████████████| 300/300 [30:52<00:00,  6.18s/it]


=== Patch Extraction Statistics ===
Amplified: 0 patches
Non_Amplified: 15 patches
ROI_Normal: 115 patches


In [9]:
# CELL 6: Load and Prepare Datasets

# Clear any cached data
gc.collect()
tf.keras.backend.clear_session()

BATCH_SIZE = 32
IMG_SIZE = (224, 224)
SEED = 123

print(f"Loading datasets from: {PATCH_ROOT}")

# Load training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    PATCH_ROOT,
    validation_split=0.30,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='int'
)

# Load validation+test dataset
temp_ds = tf.keras.utils.image_dataset_from_directory(
    PATCH_ROOT,
    validation_split=0.30,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='int'
)


# Get class names
class_names = train_ds.class_names
print(f"\nClass names: {class_names}")
print(f"Number of classes: {len(class_names)}")

# Split temp into val and test
temp_batches = tf.data.experimental.cardinality(temp_ds).numpy()
val_ds = temp_ds.take(temp_batches // 2)
test_ds = temp_ds.skip(temp_batches // 2)

# Optimize datasets
train_ds = train_ds.shuffle(1000).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

print("Datasets loaded and optimized!")

Loading datasets from: /kaggle/working/patches
Found 130 files belonging to 3 classes.
Using 91 files for training.


I0000 00:00:1766832825.041948      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Found 130 files belonging to 3 classes.
Using 39 files for validation.

Class names: ['Amplified', 'Non_Amplified', 'ROI_Normal']
Number of classes: 3
Datasets loaded and optimized!


In [10]:
# CELL 7: Calculate Class Weights


all_labels = []
for _, labels in train_ds:
    all_labels.extend(labels.numpy())

all_labels = np.array(all_labels)
print(f"\nTotal training samples: {len(all_labels)}")
print(f"Label distribution: {dict(Counter(all_labels))}")

class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(all_labels),
    y=all_labels
)

class_weight_dict = {i: w for i, w in enumerate(class_weights)}
print(f"Class weights: {class_weight_dict}")


Total training samples: 91
Label distribution: {np.int32(2): 78, np.int32(1): 13}
Class weights: {0: np.float64(3.5), 1: np.float64(0.5833333333333334)}


In [11]:
# CELL 8: Data Augmentation

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomFlip("vertical"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.15),
    tf.keras.layers.RandomContrast(0.2),
    tf.keras.layers.RandomBrightness(0.1),
], name="data_augmentation")

print("Data augmentation pipeline created!")

Data augmentation pipeline created!


In [12]:
def _iso_winsor_numpy_flat(x_batch_np, contamination=0.05, win_limit=0.05, random_state=42):
    if hasattr(x_batch_np, "numpy"):
        x_batch_np = x_batch_np.numpy()
    x_batch_np = np.asarray(x_batch_np, dtype=np.float32)
    
    B, F = x_batch_np.shape
    
    clf = IsolationForest(contamination=contamination, random_state=random_state)
    preds = clf.fit_predict(x_batch_np)
    
    out = x_batch_np.copy()
    for i in range(B):
        if preds[i] == -1:
            w = winsorize(out[i], limits=[win_limit, win_limit])
            out[i] = np.asarray(w, dtype=np.float32)
    
    return out


def isolation_winsor_layer(contamination=0.05, win_limit=0.05, random_state=42):
    def layer_op(x2d):
        @tf.custom_gradient
        def _op(x_in):
            y = tf.py_function(
                func=lambda z: _iso_winsor_numpy_flat(
                    z, contamination=contamination,
                    win_limit=win_limit, random_state=random_state
                ),
                inp=[x_in],
                Tout=tf.float32
            )
            y.set_shape(x_in.shape)
            
            def grad(dy):
                return dy
            return y, grad
        return _op(x2d)
    return layer_op

print("Robust outlier filtering layer defined!")

Robust outlier filtering layer defined!


In [13]:
# CELL 10: Squeeze-and-Excitation Block

def squeeze_excitation_block(input_tensor, ratio=16, name='se'):
    """Squeeze-and-Excitation block for channel attention."""
    from tensorflow.keras import layers
    
    channels = input_tensor.shape[-1]
    se = layers.GlobalAveragePooling2D(name=f'{name}_gap')(input_tensor)
    se = layers.Dense(channels // ratio, activation='relu', name=f'{name}_fc1')(se)
    se = layers.Dense(channels, activation='sigmoid', name=f'{name}_fc2')(se)
    se = layers.Reshape((1, 1, channels), name=f'{name}_reshape')(se)
    output = layers.Multiply(name=f'{name}_multiply')([input_tensor, se])
    return output

print("SE block function defined!")

SE block function defined!


In [15]:
# CELL 11: Build Enhanced Model

def build_enhanced_model(input_shape=(224, 224, 3), num_classes=3, 
                         use_robust_filter=True, contamination=0.05):
    
    inputs = layers.Input(shape=input_shape, name='input')
    
    x = data_augmentation(inputs)
    # ❌ REMOVE manual rescaling
    
    base_model = EfficientNetB0(
        include_top=False,
        weights="imagenet",
        input_tensor=x
    )
    base_model.trainable = False
    
    x = base_model.output
    x = squeeze_excitation_block(x, ratio=16, name='se_post_backbone')
    x = layers.GlobalAveragePooling2D(name='global_pool')(x)
    
    if use_robust_filter:
        x = layers.Lambda(
            isolation_winsor_layer(contamination=contamination, win_limit=0.05),
            name='robust_filter'
        )(x)
    
    x = layers.Dense(256, activation='relu', name='fc1')(x)
    x = layers.Dropout(0.5, name='dropout1')(x)
    x = layers.Dense(128, activation='relu', name='fc2')(x)
    x = layers.Dropout(0.3, name='dropout2')(x)
    
    outputs = layers.Dense(num_classes, activation='softmax', name='output')(x)
    
    model = models.Model(inputs, outputs, name='HER2_SISH_Classifier')
    return model



# Build the model
model = build_enhanced_model(
    input_shape=(224, 224, 3),
    num_classes=len(class_names),
    use_robust_filter=True,
    contamination=0.05
)

print("Model built successfully!")


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Model built successfully!


In [19]:
# CELL 12: Compile Model

# Disable problematic TensorFlow optimizers
tf.config.optimizer.set_experimental_options({
    'layout_optimizer': False,
})

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)



print("Model compiled!")

Model compiled!


In [20]:
# CELL 13: Define Training Callbacks

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=8,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=4,
        min_lr=1e-7,
        verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'best_model.keras',
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        verbose=1
    )
]

print("Callbacks defined!")

Callbacks defined!


In [21]:
# CELL 14: Training Phase 1 - Frozen Backbone

print("Phase 1: Training with Frozen Backbone")

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25,
    callbacks=callbacks,
    class_weight=class_weight_dict,
    verbose=1
)

print("Phase 1 training completed!")

Phase 1: Training with Frozen Backbone
Epoch 1/25


ValueError: Shape must be at least rank 1 but is rank 0 for '{{node loop_body/UnsortedSegmentSum}} = UnsortedSegmentSum[T=DT_FLOAT, Tindices=DT_INT32, Tnumsegments=DT_INT32](loop_body/GatherV2, loop_body/GatherV2_1, loop_body/UnsortedSegmentSum/num_segments)' with input shapes: [], [?], [].

In [28]:
# CELL 15: Training Phase 2 - Fine-tuning

print("Phase 2: Fine-tuning")

# Unfreeze last blocks of EfficientNet
for layer in model.layers:
    if isinstance(layer, tf.keras.Model):
        for sublayer in layer.layers:
            if 'block6' in sublayer.name or 'block7' in sublayer.name:
                sublayer.trainable = True

# Recompile with lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc', multi_label=False)]
)

fine_history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=callbacks,
    class_weight=class_weight_dict,
    verbose=1
)

print("Phase 2 training completed!")

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [29]:
# CELL 16: Plot Training History

def plot_training_history(history, fine_history):
    metrics = ['loss', 'accuracy', 'auc']
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    for idx, metric in enumerate(metrics):
        train_metric = history.history[metric] + fine_history.history[metric]
        val_metric = history.history[f'val_{metric}'] + fine_history.history[f'val_{metric}']
        
        epochs = range(1, len(train_metric) + 1)
        
        axes[idx].plot(epochs, train_metric, 'b-o', label=f'Training', alpha=0.8)
        axes[idx].plot(epochs, val_metric, 'r--s', label=f'Validation', alpha=0.8)
        axes[idx].axvline(x=len(history.history[metric]), color='g', 
                         linestyle=':', label='Fine-tuning Start')
        axes[idx].set_title(f'{metric.upper()}')
        axes[idx].set_xlabel('Epoch')
        axes[idx].set_ylabel(metric.capitalize())
        axes[idx].legend()
        axes[idx].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

plot_training_history(history, fine_history)

In [32]:
def predict_with_tta(model, images, num_augmentations=5):
    predictions = []
    
    for _ in range(num_augmentations):
        aug_images = data_augmentation(images, training=True)
        preds = model.predict(aug_images, verbose=0)
        predictions.append(preds)
    
    avg_predictions = np.mean(predictions, axis=0)
    return avg_predictions

print("TTA function defined!")

Epoch 1/25


E0000 00:00:1766824973.835076      55 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/HER2_SISH_Classifier_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


InvalidArgumentError: Graph execution error:

Detected at node GatherV2 defined at (most recent call last):
<stack traces unavailable>
Detected at node GatherV2 defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) INVALID_ARGUMENT:  Error in user-defined function passed to MapDataset:22 transformation with iterator: Iterator::Root::Prefetch::ParallelMapV2: indices[0] = 2 is not in [0, 2)
	 [[{{node GatherV2}}]]
	 [[IteratorGetNext]]
	 [[IteratorGetNext/_2]]
  (1) INVALID_ARGUMENT:  Error in user-defined function passed to MapDataset:22 transformation with iterator: Iterator::Root::Prefetch::ParallelMapV2: indices[0] = 2 is not in [0, 2)
	 [[{{node GatherV2}}]]
	 [[IteratorGetNext]]
0 successful operations.
0 derived errors ignored. [Op:__inference_multi_step_on_iterator_51518]

In [ ]:
# CELL 18: MIL (Multiple Instance Learning) Evaluation

def mil_topk_with_tta(model, dataset, k=10, use_tta=True):
    slide_probs = defaultdict(list)
    slide_labels = {}
    
    patch_idx = 0
    for images, labels in tqdm(dataset, desc="MIL Inference"):
        if use_tta:
            preds = predict_with_tta(model, images, num_augmentations=5)
        else:
            preds = model.predict(images, verbose=0)
        
        for i, pred in enumerate(preds):
            slide_id = patch_idx // 20
            slide_probs[slide_id].append(pred)
            slide_labels[slide_id] = labels[i].numpy()
            patch_idx += 1
    
    y_true, y_pred, y_probs = [], [], []
    for slide_id in sorted(slide_probs.keys()):
        probs = slide_probs[slide_id]
        confidences = [np.max(p) for p in probs]
        top_indices = np.argsort(confidences)[-k:]
        top_probs = [probs[i] for i in top_indices]
        mean_prob = np.mean(top_probs, axis=0)
        pred_class = np.argmax(mean_prob)
        
        y_true.append(slide_labels[slide_id])
        y_pred.append(pred_class)
        y_probs.append(mean_prob)
    
    return np.array(y_true), np.array(y_pred), np.array(y_probs)

print("MIL function defined!")

In [ ]:
print("Evaluating on Test Set with MIL + TTA")

y_true, y_pred, y_probs = mil_topk_with_tta(model, test_ds, k=10, use_tta=True)

print("\nClassification Report")
print(classification_report(y_true, y_pred, target_names=class_names, digits=3))


In [ ]:
# CELL 20: Plot Confusion Matrix

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix - HER2-SISH Classification', fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nEvaluation completed!")

In [ ]:
# CELL 22: Generate Summary Report
print("FINAL SUMMARY REPORT")

print(f"\nDataset Information:")
print(f"  - Total classes: {len(class_names)}")
print(f"  - Class names: {class_names}")
print(f"  - Training samples: {len(all_labels)}")
print(f"  - Class distribution: {dict(Counter(all_labels))}")

print(f"\nModel Configuration:")
print(f"  - Architecture: EfficientNetB0 with SE blocks")
print(f"  - Input shape: (224, 224, 3)")
print(f"  - Normalization: {NORMALIZATION_METHOD}")
print(f"  - Robust filtering: Enabled")
print(f"  - Data augmentation: Enabled")

print(f"\nTraining Configuration:")
print(f"  - Batch size: {BATCH_SIZE}")
print(f"  - Phase 1 epochs: {len(history.history['loss'])}")
print(f"  - Phase 2 epochs: {len(fine_history.history['loss'])}")
print(f"  - Class weights: {class_weight_dict}")

print(f"\nFinal Performance:")
print(f"  - Test accuracy: {np.mean(y_true == y_pred):.3f}")
print(f"  - Per-class accuracy:")
for i, class_name in enumerate(class_names):
    class_mask = y_true == i
    if np.sum(class_mask) > 0:
        class_acc = np.mean(y_pred[class_mask] == i)
        print(f"    {class_name}: {class_acc:.3f}")

print("Pipeline completed successfully! ✅")


In [ ]:
# Unfreeze base model for fine-tuning
for layer in model.layers:
    if "inception_resnet_v2" in layer.name:
        layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-6), # Extremely low LR
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

fine_history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=callbacks
)

In [16]:
num_classes = 3

model = build_robust_model(
    input_shape=(224, 224, 3),
    num_classes=num_classes,
    use_robust_filter=True,
    contamination=0.05
)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# Display model summary


In [14]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=8,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=4,
        min_lr=1e-7,
        verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'best_inception_model.keras',
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        verbose=1
    )
]


In [15]:
# Training Phase 1: Frozen backbone
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25,
    callbacks=callbacks,
    # class_weight=class_weight_dict,  # Uncomment if you have class weights
    verbose=1
)

# Training Phase 2: Fine-tuning
print("\n=== Phase 2: Fine-tuning ===")

# Unfreeze the last few layers of InceptionResNetV2
for layer in model.layers:
    if isinstance(layer, tf.keras.Model):  # The InceptionResNetV2 base
        # Unfreeze from 'mixed_7a' onwards (last few blocks)
        unfreeze = False
        for sublayer in layer.layers:
            if 'mixed_7a' in sublayer.name:
                unfreeze = True
            if unfreeze:
                sublayer.trainable = True

# Recompile with lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# Continue training
fine_history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=callbacks,
    # class_weight=class_weight_dict,
    verbose=1
)

NameError: name 'train_ds' is not defined